# Markov-Chains Multi-Touch Attribution (Portfolio Demo)

This notebook builds a **first-order Markov chain** over GTM **channels** to estimate each channel's contribution via the **removal effect** method.

**Pipeline:**
1. Load closed-won opportunities and their pre-conversion touch sequences (within lookback window).
2. Build transitions including `START` and absorbing states `CONVERT`, `NULL`.
3. Estimate transition probabilities `P(channel_j | channel_i)`.
4. Compute baseline conversion probability, then **remove** each channel and recompute; the delta is the channel's **Markov attribution**.

> Dataset comes from `warehouse/gtm.db` built by `python -m etl.run_pipeline`.


In [ ]:
from pathlib import Path
import sqlite3
import pandas as pd
import numpy as np

ROOT = Path(__file__).resolve().parents[1]
DB = ROOT/'warehouse'/'gtm.db'
assert DB.exists(), 'Run: python -m etl.run_pipeline'
con = sqlite3.connect(DB.as_posix())

# Load touches and won opps
touches = pd.read_sql_query('SELECT lead_id, channel, campaign, ts FROM touches ORDER BY ts', con, parse_dates=['ts'])
opps = pd.read_sql_query('SELECT opp_id, lead_id, amount, created_at, closed_at FROM opportunities WHERE is_closed_won=1', con, parse_dates=['created_at','closed_at'])
con.close()
len(touches), len(opps)

In [ ]:
# Build channel sequences per won opportunity within a 60-day lookback before close
LOOKBACK_DAYS = 60
seqs = []
for _, o in opps.iterrows():
    lb = o['closed_at'] - pd.Timedelta(days=LOOKBACK_DAYS)
    t = touches[(touches.lead_id==o.lead_id) & (touches.ts>=lb) & (touches.ts<=o.closed_at)].sort_values('ts')
    if t.empty: continue
    seqs.append(['START'] + t['channel'].tolist() + ['CONVERT'])
len(seqs), seqs[0][:6] if seqs else []

In [ ]:
# Count transitions
from collections import defaultdict
edges = defaultdict(int)
nodes = set()
for s in seqs:
    for a,b in zip(s[:-1], s[1:]):
        edges[(a,b)] += 1
        nodes.add(a); nodes.add(b)
nodes = sorted(nodes)
len(edges), list(edges.items())[:5]

In [ ]:
# Transition matrix P(b|a)
import pandas as pd
out_edges = {}
for (a,b), c in edges.items():
    out_edges.setdefault(a, 0)
    out_edges[a] += c
P = {}
for (a,b), c in edges.items():
    P[(a,b)] = c / out_edges[a]
P_df = pd.DataFrame([{'from':a, 'to':b, 'p':p} for (a,b), p in P.items()])
P_df.sort_values(['from','to']).head(10)

In [ ]:
# Compute baseline conversion probability from START via simulation (or absorbing Markov chain math)
rng = np.random.default_rng(7)
def step(state):
    # sample next state from P
    cand = [(b, p) for (a,b), p in P.items() if a==state]
    if not cand:
        return 'NULL'
    to, probs = zip(*cand)
    return rng.choice(to, p=probs)

def simulate(n=20000, max_steps=50):
    conv = 0
    for _ in range(n):
        s = 'START'
        for _ in range(max_steps):
            s = step(s)
            if s in ('CONVERT','NULL'):
                conv += int(s=='CONVERT')
                break
    return conv / n

baseline = simulate(10000)
baseline

In [ ]:
# Removal effect: for each channel, remove its outgoing transitions and re-normalize, then recompute conversion rate
channels = [n for n in nodes if n not in ('START','CONVERT','NULL')]
def removal_effect(channel):
    # build modified P without the channel
    out = {}
    totals = {}
    for (a,b), p in P.items():
        if a==channel or b==channel:
            continue
        out[(a,b)] = p
        totals.setdefault(a, 0)
        totals[a] += p
    # renormalize
    P2 = {}
    for (a,b), p in out.items():
        P2[(a,b)] = p / totals[a] if totals.get(a,0)>0 else 0
    # simulate
    rng2 = np.random.default_rng(9)
    def step2(state):
        cand = [(b, pr) for (x,b), pr in P2.items() if x==state]
        if not cand:
            return 'NULL'
        to, probs = zip(*cand)
        return rng2.choice(to, p=probs)
    conv = 0
    for _ in range(8000):
        s='START'
        for _ in range(50):
            cand = [(b, pr) for (x,b), pr in P2.items() if x==s]
            if not cand:
                s='NULL'; break
            s = step2(s)
            if s in ('CONVERT','NULL'):
                conv += int(s=='CONVERT'); break
    return conv/8000

rows=[]
for ch in channels:
    conv_rate = removal_effect(ch)
    rows.append({'channel': ch, 'baseline_conv': baseline, 'conv_without': conv_rate, 'delta': baseline - conv_rate})
markov_attr = pd.DataFrame(rows).sort_values('delta', ascending=False)
markov_attr.head(10)

## Compare with heuristic models (optional)
You can compare `markov_attr` with the heuristic attributions in the warehouse (`touch_attribution` table) to see alignment.
